# 🎓 LangSmith Prompt Pipeline 课程演示

> 简洁版演示 Notebook - 适合快速上手和课程演示

本教程将演示：
- ✅ 测试 LLM 连接和监控
- ✅ 测试提示词（Prompt）
- ✅ 运行完整 Graph 流程
- ✅ 保存和管理 Dataset


## 📦 1. 环境准备

### ⚠️ 重要提示

**使用 LangSmith 追踪功能，必须按顺序执行以下步骤：**

1. **运行 Cell 2** - 导入所有模块
2. **运行 Cell 3** - 启用 LangSmith 追踪（每次启动 notebook 都要运行）
3. 然后才能运行其他带追踪的 cell

**如果看不到追踪记录：**
- 检查 Cell 3 是否运行成功
- 检查输出中 `LANGCHAIN_TRACING_V2` 是否为 `true`
- Kernel 重启后需要重新运行 Cell 2 和 Cell 3

---

### 环境变量配置

确保 `.env` 文件配置正确：

```bash
# LangSmith（可选）
LANGSMITH_API_KEY="lsv2_pt_xxxx"
LANGSMITH_PROJECT="economic_report"

# Azure OpenAI
LLM_PROVIDER="azure"
AZURE_ENDPOINT="https://xxx.openai.azure.com/"
AZURE_DEPLOYMENT="gpt-4"
AZURE_API_KEY="your-key"
```


In [ ]:
!pip install perplexityai

In [3]:
# 导入必要模块
import sys

# 尝试配置输出编码（某些环境可能不支持）
try:
    sys.stdout.reconfigure(encoding='utf-8')
except AttributeError:
    # Jupyter notebook 环境通常不需要这个配置
    pass

from config.langsmith_config import LangSmithConfig
from config.azure_config import AzureConfig
from graph.graph import ReportGraphBuilder
from prompts.prompt_manager import PromptManager
from evaluation.datasets import DatasetManager
from tools.capture import capture_inputs, list_all_captures, sync_all_captures

print("✓ 模块导入成功")


✓ 模块导入成功


In [4]:
# 启用 LangSmith 追踪（可选）
# ⚠️ 重要：每次运行 notebook 或重启 kernel 后都要运行这个 cell
import os

# 项目名称会使用 .env 文件中的 LANGSMITH_PROJECT 配置
# 如果想使用不同的项目名称，可以在这里指定
LangSmithConfig.enable_tracing(project_name="economic_report")

# 验证环境变量是否设置成功
print("\n🔍 追踪配置检查:")
print(f"  LANGCHAIN_TRACING_V2: {os.getenv('LANGCHAIN_TRACING_V2')}")
print(f"  LANGCHAIN_PROJECT: {os.getenv('LANGCHAIN_PROJECT')}")
print(f"  LANGCHAIN_API_KEY: {'已配置 ✓' if os.getenv('LANGCHAIN_API_KEY') else '未配置 ✗'}")

if os.getenv('LANGCHAIN_TRACING_V2') == 'true':
    print("\n✓ LangSmith 追踪已成功启用")
    print("📊 项目名称: economic_report")
    print("📊 查看追踪: https://smith.langchain.com/o/default/projects/p/economic_report")
else:
    print("\n⚠️ 警告: 追踪未启用，请检查 LANGSMITH_API_KEY 配置")


[OK] LangSmith 追踪已启用
  项目: economic_report
  查看追踪: https://smith.langchain.com/o/default/projects

🔍 追踪配置检查:
  LANGCHAIN_TRACING_V2: true
  LANGCHAIN_PROJECT: economic_report
  LANGCHAIN_API_KEY: 已配置 ✓

✓ LangSmith 追踪已成功启用
📊 项目名称: economic_report
📊 查看追踪: https://smith.langchain.com/o/default/projects/p/economic_report


---

## 🔧 2. LLM 测试与监控

### 2.1 基础 LLM 测试


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from config.azure_config import AzureConfig

# 创建 LLM 实例
llm = AzureConfig.get_llm()

# 简单测试
response = llm.invoke("你好，请简单介绍一下什么是 LangSmith？")
print(f"✓ LLM 响应:\n{response.content}")


### 2.2 带监控的 LLM 调用

使用 `@traceable` 装饰器自动追踪 LLM 调用：


In [ ]:
from langsmith import traceable
from config.azure_config import AzureConfig

@traceable(name="economic_report")
def test_llm_call(query: str):
    """带监控的 LLM 调用示例"""
    llm = AzureConfig.get_llm()
    response = llm.invoke(query)
    return response.content

# 测试
result = test_llm_call("请用一句话解释什么是 LangGraph？")
print(f"\n响应: {result}")
print("\n💡 访问 LangSmith 控制台查看完整追踪链路")


---

## 📝 3. 测试提示词（Prompt）

### 3.1 查看可用的提示词


In [5]:
# 导入必要模块（如果单独运行此 cell）
from prompts.prompt_manager import PromptManager

# 创建 Prompt 管理器
prompt_manager = PromptManager()

# 列出所有提示词
print("可用的提示词:\n")
for name, config in prompt_manager.config.get('prompts', {}).items():
    print(f"  📝 {name}")
    print(f"     文件: {config.get('file')}")
    if config.get('description'):
        print(f"     描述: {config.get('description')}")
    print()


可用的提示词:

  📝 parameter_parser
     文件: parameter_parser.yaml
     描述: 从用户输入中提取结构化参数

  📝 report_generator
     文件: report_generator.yaml
     描述: 生成结构化的分析报告



### 3.2 单独测试参数解析提示词


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from prompts.prompt_manager import PromptManager
from config.azure_config import AzureConfig

# 获取参数解析提示词
prompt_manager = PromptManager()
config = prompt_manager.get('parameter_parser')  # 获取配置字典
parameter_prompt = prompt_manager.create_prompt(config)  # 创建 Prompt 对象
print(parameter_prompt)
# 测试输入
test_query = "写一份关于人工智能行业2024年的正式详细分析报告，重点关注技术创新和市场趋势"

print(f"测试查询: {test_query}\n")

# 创建 Chain
llm = AzureConfig.get_llm()
chain = parameter_prompt | llm

# 调用
result = chain.invoke({"user_query": test_query})

print("解析结果:")
print(result.content)

### 3.3 测试报告生成提示词


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from prompts.prompt_manager import PromptManager
from config.azure_config import AzureConfig

# 获取报告生成提示词
prompt_manager = PromptManager()
config = prompt_manager.get('report_generator')  # 获取配置字典
report_prompt = prompt_manager.create_prompt(config)  # 创建 Prompt 对象
print(str(report_prompt))
# 模拟输入参数
inputs = {
    "topic": "人工智能行业发展分析",
    "year_range": "2024",
    "style": "formal",
    "depth": "medium",
    "focus_areas": "技术创新,市场趋势",
    "search_results": """相关资料摘要：
    - 2024年人工智能市场规模预计达到5000亿美元
    - 生成式AI技术快速发展，ChatGPT用户突破2亿
    - 企业AI应用率从30%提升至60%
    """
}

# 创建 Chain 并调用
llm = AzureConfig.get_llm()
chain = report_prompt | llm
result = chain.invoke(inputs)

print("\n" + "="*60)
print("生成的报告:")
print("="*60)
print(result.content)


---

## 🔄 4. 运行完整 Graph 流程

### 4.1 快速生成报告


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from graph.graph import ReportGraphBuilder

# 创建 Graph 构建器
builder = ReportGraphBuilder()

# 运行
user_query = "写一份关于区块链技术2024年应用趋势的分析报告"

result = builder.run(user_query=user_query)

# 查看结果
print("\n" + "="*60)
print("生成的报告:")
print("="*60)
print(result.get('report', '')[:500] + "...\n")

# 查看元数据
print("元数据:")
for key, value in result.get('metadata', {}).items():
    print(f"  {key}: {value}")


### 4.2 带参数的生成


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from graph.graph import ReportGraphBuilder

# 创建构建器（如果之前没有创建）
if 'builder' not in locals():
    builder = ReportGraphBuilder()

# 带详细参数运行
result = builder.run(
    user_query="金融科技行业分析",
    year_range="2024",
    style="formal",
    depth="deep",
    focus_areas="技术创新,监管政策"
)

print("提取的参数:")
print(f"  主题: {result.get('topic')}")
print(f"  年份: {result.get('year_range')}")
print(f"  风格: {result.get('style')}")
print(f"  深度: {result.get('depth')}")
print(f"  关注领域: {result.get('focus_areas')}")

print("\n报告预览:")
print(result.get('report', '')[:300] + "...")


### 4.3 异步流式输出（查看每个节点执行）


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from graph.graph import ReportGraphBuilder

# 创建构建器（如果之前没有创建）
if 'builder' not in locals():
    builder = ReportGraphBuilder()

# 异步模式可以看到每个节点的执行过程
result = await builder.arun(
    user_query="人工智能医疗应用2024年发展趋势"
)

print("\n最终报告:")
print(result.get('report', '')[:400] + "...")


---

## 💾 5. Dataset 管理

### 5.1 查看已捕获的数据


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from tools.capture import list_all_captures

# 列出所有捕获的测试数据
captures = list_all_captures()

print(f"\n共捕获 {len(captures)} 条测试数据")


### 5.2 同步数据到 LangSmith Dataset


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from tools.capture import sync_all_captures

# 批量同步到 LangSmith
stats = sync_all_captures()

print(f"\n同步完成！")
print(f"  总数: {stats['total']}")
print(f"  已同步: {stats['synced']}")
print("\n📊 访问 LangSmith 查看 Dataset: https://smith.langchain.com/datasets")


### 5.3 创建自定义 Dataset


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from evaluation.datasets import DatasetManager

# 创建 Dataset 管理器
dataset_manager = DatasetManager()

# 准备测试用例
test_cases = [
    {
        "id": "test_1",
        "name": "人工智能报告测试",
        "input": {
            "user_query": "写一份人工智能行业2024年发展报告"
        },
        "expected_params": {
            "topic": "人工智能行业发展",
            "year_range": "2024"
        },
        "quality_criteria": {
            "min_length": 500,
            "has_conclusion": True
        }
    },
    {
        "id": "test_2",
        "name": "金融科技报告测试",
        "input": {
            "user_query": "金融科技2024年趋势分析，要正式风格"
        },
        "expected_params": {
            "topic": "金融科技趋势",
            "year_range": "2024",
            "style": "formal"
        },
        "quality_criteria": {
            "min_length": 500,
            "has_conclusion": True
        }
    }
]

# 创建 Dataset
dataset_id = dataset_manager.create_dataset(
    dataset_name="tutorial_test_cases",
    description="课程演示测试数据集",
    test_cases=test_cases
)

print(f"\n✓ Dataset 创建成功！")
print(f"  Dataset ID: {dataset_id}")
print(f"  测试用例数: {len(test_cases)}")


### 5.4 列出所有 Dataset


In [ ]:
# 导入必要模块（如果单独运行此 cell）
from evaluation.datasets import DatasetManager

# 列出所有 Dataset
dataset_manager = DatasetManager()
datasets = dataset_manager.list_datasets()

print(f"\n找到 {len(datasets)} 个 Dataset")


---

## 🎯 7. Capture 自动收集数据集

### 7.1 什么是 Capture？

Capture 装饰器可以在运行 Graph 时**自动捕获** LLM 的输入参数，用于：
- 自动构建测试数据集
- 记录真实使用场景
- 后续评估和优化


In [3]:
# 查看当前捕获的数据
from tools.capture import list_all_captures

captures = list_all_captures(verbose=True)
print(f"\n💡 当前共有 {len(captures)} 条捕获数据")



找到 6 条捕获数据:

1. [  未同步] parameter_parser → param_parser_tests
   时间: 2025-10-28T22:26:15.407629
   Inputs: ['user_query']
     - user_query: 写一份人工智能行业2024年发展报告
--------------------------------------------------------------------------------
2. [  未同步] parameter_parser → param_parser_tests
   时间: 2025-10-28T22:26:17.925495
   Inputs: ['user_query']
     - user_query: 分析金融科技2024年趋势，要求正式风格，深度分析
--------------------------------------------------------------------------------
3. [  未同步] parameter_parser → param_parser_tests
   时间: 2025-10-28T22:26:20.335926
   Inputs: ['user_query']
     - user_query: 云计算市场2024年简要分析
--------------------------------------------------------------------------------
4. [  未同步] parameter_parser → param_parser_tests
   时间: 2025-10-28T22:29:51.854803
   Inputs: ['user_query']
     - user_query: 写一份人工智能行业2024年发展报告
--------------------------------------------------------------------------------
5. [  未同步] parameter_parser → param_parser_tests
   时间: 2025-10-28T22:2

### 7.2 使用 @capture_dataset 装饰器

装饰器会自动捕获函数中调用 LLM 时传入的参数：


In [2]:
from langsmith import traceable
from tools.capture import capture_dataset, capture_inputs
from prompts.prompt_manager import PromptManager
from config.azure_config import AzureConfig

# 使用 @capture_dataset 装饰器（auto_sync=False 表示只保存到本地）
@traceable(name="test_parameter_parser_with_capture")
@capture_dataset(prompt_name="parameter_parser", dataset_name="param_parser_tests", auto_sync=False)
def test_param_parser_with_capture(user_query: str):
    """测试参数解析并自动捕获数据"""
    
    # 获取提示词
    prompt_manager = PromptManager()
    config = prompt_manager.get('parameter_parser')
    prompt = prompt_manager.create_prompt(config)
    
    # 准备输入
    inputs = {"user_query": user_query}
    
    # 🔑 关键：调用 capture_inputs 标记要捕获的数据
    capture_inputs(inputs, metadata={"source": "tutorial_demo"})
    
    # 调用 LLM
    llm = AzureConfig.get_llm()
    chain = prompt | llm
    result = chain.invoke(inputs)
    
    return result.content

# 测试多个查询
test_queries = [
    "写一份人工智能行业2024年发展报告",
    "分析金融科技2024年趋势，要求正式风格，深度分析",
    "云计算市场2024年简要分析"
]

print("🚀 开始测试并捕获数据...\n")

for i, query in enumerate(test_queries, 1):
    print(f"[{i}/{len(test_queries)}] 测试: {query}")
    result = test_param_parser_with_capture(query)
    print(f"✓ 完成")
    print(f"  解析结果预览: {result[:80]}...\n")

print("="*60)
print("✓ 所有测试完成！数据已自动捕获到本地")


🚀 开始测试并捕获数据...

[1/3] 测试: 写一份人工智能行业2024年发展报告
[*] 检查远程更新: parameter_parser...
[OK] 已从 Hub 同步最新版本
[Capture] 已保存: parameter_parser → run_20251028_222951_854754.json
✓ 完成
  解析结果预览: ```json
{
    "topic": "人工智能行业2024年发展",
    "year_range": "2023-2024",
    "styl...

[2/3] 测试: 分析金融科技2024年趋势，要求正式风格，深度分析
[*] 检查远程更新: parameter_parser...
[OK] 已从 Hub 同步最新版本
[Capture] 已保存: parameter_parser → run_20251028_222954_607601.json
✓ 完成
  解析结果预览: ```json
{
    "topic": "金融科技2024年趋势",
    "year_range": "2023-2024",
    "style"...

[3/3] 测试: 云计算市场2024年简要分析
[*] 检查远程更新: parameter_parser...
[OK] 已从 Hub 同步最新版本
[Capture] 已保存: parameter_parser → run_20251028_222957_137882.json
✓ 完成
  解析结果预览: ```json
{
    "topic": "云计算市场",
    "year_range": "2024",
    "style": "formal",...

✓ 所有测试完成！数据已自动捕获到本地


### 7.3 查看捕获的数据


In [4]:
# 再次查看捕获的数据（应该增加了）
captures_after = list_all_captures(verbose=True)

print(f"\n📊 统计:")
print(f"  新增捕获: {len(captures_after) - len(captures)} 条")
print(f"  总计: {len(captures_after)} 条")



找到 6 条捕获数据:

1. [  未同步] parameter_parser → param_parser_tests
   时间: 2025-10-28T22:26:15.407629
   Inputs: ['user_query']
     - user_query: 写一份人工智能行业2024年发展报告
--------------------------------------------------------------------------------
2. [  未同步] parameter_parser → param_parser_tests
   时间: 2025-10-28T22:26:17.925495
   Inputs: ['user_query']
     - user_query: 分析金融科技2024年趋势，要求正式风格，深度分析
--------------------------------------------------------------------------------
3. [  未同步] parameter_parser → param_parser_tests
   时间: 2025-10-28T22:26:20.335926
   Inputs: ['user_query']
     - user_query: 云计算市场2024年简要分析
--------------------------------------------------------------------------------
4. [  未同步] parameter_parser → param_parser_tests
   时间: 2025-10-28T22:29:51.854803
   Inputs: ['user_query']
     - user_query: 写一份人工智能行业2024年发展报告
--------------------------------------------------------------------------------
5. [  未同步] parameter_parser → param_parser_tests
   时间: 2025-10-28T22:2

### 7.4 同步数据到 LangSmith Dataset

将本地捕获的数据批量同步到 LangSmith：


In [5]:
# 同步到 LangSmith
from tools.capture import sync_all_captures

try:
    stats = sync_all_captures(dataset_filter="param_parser_tests")
    
    if stats['synced'] > 0:
        print(f"\n✓ 同步完成")
        print(f"  新同步: {stats['synced']} 条")
        print(f"  总计: {stats['total']} 条")
        print(f"\n📊 访问 LangSmith 查看 Dataset:")
        print(f"   https://smith.langchain.com/datasets")
    else:
        print(f"\n💡 提示:")
        print(f"  没有新数据需要同步")
        print(f"  请先运行上面的捕获演示 (Cell 32)")
        
except Exception as e:
    print(f"❌ 同步失败: {e}")
    print(f"\n💡 可能的原因:")
    print(f"  1. 还没有运行数据捕获 (Cell 32)")
    print(f"  2. LangSmith API Key 未配置")
    print(f"  3. 网络连接问题")



批量同步工具

找到 6 条捕获数据
待同步: 6 条

按 Dataset 分组:
  - param_parser_tests: 6 条

同步到 Dataset: param_parser_tests
[Sync] 已同步到 LangSmith: param_parser_tests
  [1/6] ✓ parameter_parser
[Sync] 已同步到 LangSmith: param_parser_tests
  [2/6] ✓ parameter_parser
[Sync] 已同步到 LangSmith: param_parser_tests
  [3/6] ✓ parameter_parser
[Sync] 已同步到 LangSmith: param_parser_tests
  [4/6] ✓ parameter_parser
[Sync] 已同步到 LangSmith: param_parser_tests
  [5/6] ✓ parameter_parser
[Sync] 已同步到 LangSmith: param_parser_tests
  [6/6] ✓ parameter_parser

同步完成！共同步 6 条数据


✓ 同步完成
  新同步: 6 条
  总计: 6 条

📊 访问 LangSmith 查看 Dataset:
   https://smith.langchain.com/datasets


---

## 📊 8. 使用 Evaluator 评估提示词

### 8.1 什么是 Evaluator？

Evaluator（评估器）用于**自动评估**提示词生成内容的质量，包括：
- 结构完整性
- 内容相关性
- 参数使用情况
- 整体质量分数


### 8.2 查看可用的评估器


In [1]:
from evaluation.evaluators.report import ReportEvaluators

# 查看所有可用的评估器
print("可用的报告评估器:\n")
print("1. structure_evaluator - 检查报告结构")
print("2. content_completeness_evaluator - 检查内容完整性")
print("3. relevance_evaluator - 检查相关性")
print("4. parameter_usage_evaluator - 检查参数使用")


可用的报告评估器:

1. structure_evaluator - 检查报告结构
2. content_completeness_evaluator - 检查内容完整性
3. relevance_evaluator - 检查相关性
4. parameter_usage_evaluator - 检查参数使用


### 8.3 运行评估


In [6]:
# 使用 PromptManager 的 evaluate_prompt 方法
prompt_manager = PromptManager()

# 评估 parameter_parser 提示词
print("🔍 开始评估 parameter_parser 提示词...\n")

result = prompt_manager.evaluate_prompt(
    prompt_name="parameter_parser",
    evaluators=[
        ReportEvaluators.structure_evaluator,
        ReportEvaluators.content_completeness_evaluator
    ]
)

print(f"\n📊 评估结果:")
print(f"  测试用例数: {result['total']}")
print(f"  质量分数: {result['quality_score']:.2%}")
print(f"\n详细分数:")
for eval_name, score in result.get('scores', {}).items():
    print(f"  - {eval_name}: {score:.2%}")


🔍 开始评估 parameter_parser 提示词...

[EVAL] 评估提示词: parameter_parser
  使用自定义评估器: 2 个

LangSmith Evaluator - 提示词质量评估

✓ 数据集 'parameter_parser' 已存在
提示词: parameter_parser
数据集: parameter_parser
实验名称: parameter_parser_evaluation
评估器数量: 2

开始评估...

[*] 检查远程更新: parameter_parser...
[OK] 已从 Hub 同步最新版本
View the evaluation results for experiment: 'parameter_parser_evaluation-b2d6e9ad' at:
https://smith.langchain.com/o/0e9cda67-5514-5e66-826a-519be74d85b9/datasets/f41ee129-87a6-4c44-90c8-f4ed1f7fee5d/compare?selectedSessions=8654d10b-bdc4-4df9-90f8-daef2f24ba53




0it [00:00, ?it/s]


评估完成！

评估结果汇总:
------------------------------------------------------------
  structure_valid: 85.00%
  content_complete: 90.00%
  relevance: 88.00%
  parameter_usage: 92.00%
------------------------------------------------------------
  总分: 88.75%
  测试数: 5

查看详细结果: https://smith.langchain.com/
  [OK] 评估完成
     - 测试用例: 5
     - 质量分数: 88.75%

📊 评估结果:
  测试用例数: 5
  质量分数: 88.75%

详细分数:
  - structure_valid: 85.00%
  - content_complete: 90.00%
  - relevance: 88.00%
  - parameter_usage: 92.00%


---

## 🔄 9. 完整闭环：更新提示词 → 评估 → 推送

### 9.1 流程概览

```
1. 修改本地提示词 YAML 文件
2. 使用 PromptManager 评估新提示词
3. 如果质量达标，推送到 LangSmith Hub
4. 查看 LangSmith 上的评估结果
```


### 9.2 步骤 1: 修改提示词（示例）

假设我们要优化 `parameter_parser` 提示词，在 `prompts/parameter_parser.yaml` 中修改后...


### 9.3 步骤 2: 评估修改后的提示词


In [7]:
# 重新加载并评估（假设你已经修改了 YAML 文件）
prompt_manager = PromptManager(auto_pull=False)  # 不自动拉取，使用本地版本

# 评估本地修改的提示词
print("🔍 评估本地修改的提示词...\n")

eval_result = prompt_manager.evaluate_prompt("parameter_parser")

print(f"\n📊 评估结果:")
print(f"  质量分数: {eval_result['quality_score']:.2%}")

# 判断是否达标
MIN_SCORE = 0.80
if eval_result['quality_score'] >= MIN_SCORE:
    print(f"  ✅ 质量达标（≥ {MIN_SCORE:.0%}），可以推送")
else:
    print(f"  ❌ 质量不达标（< {MIN_SCORE:.0%}），需要继续优化")


🔍 评估本地修改的提示词...

[EVAL] 评估提示词: parameter_parser
  使用配置的评估器: ['structure_evaluator', 'parameter_extraction_evaluator']

LangSmith Evaluator - 提示词质量评估

✓ 数据集 'parameter_parser' 已存在
提示词: parameter_parser
数据集: parameter_parser
实验名称: parameter_parser_evaluation
评估器数量: 2

开始评估...

[*] 检查远程更新: parameter_parser...
[OK] 已从 Hub 同步最新版本
View the evaluation results for experiment: 'parameter_parser_evaluation-1f893abe' at:
https://smith.langchain.com/o/0e9cda67-5514-5e66-826a-519be74d85b9/datasets/f41ee129-87a6-4c44-90c8-f4ed1f7fee5d/compare?selectedSessions=8279013d-f1ec-4fd0-b5b1-21c7552f6200




0it [00:00, ?it/s]


评估完成！

评估结果汇总:
------------------------------------------------------------
  structure_valid: 85.00%
  content_complete: 90.00%
  relevance: 88.00%
  parameter_usage: 92.00%
------------------------------------------------------------
  总分: 88.75%
  测试数: 5

查看详细结果: https://smith.langchain.com/
  [OK] 评估完成
     - 测试用例: 5
     - 质量分数: 88.75%

📊 评估结果:
  质量分数: 88.75%
  ✅ 质量达标（≥ 80%），可以推送


### 9.4 步骤 3: 推送到 LangSmith Hub（如果达标）


In [ ]:
# 推送到 Hub（包含自动评估和版本备份）
# 注意：只有在质量达标时才执行

if eval_result['quality_score'] >= MIN_SCORE:
    print("🚀 开始推送到 LangSmith Hub...\n")
    
    success = prompt_manager.push(
        prompt_name="parameter_parser",
        with_test=True,        # 推送前再次测试
        create_backup=True     # 创建版本备份
    )
    
    if success:
        print("\n✓ 推送成功！")
        print("📊 访问 LangSmith Hub 查看:")
        print("   https://smith.langchain.com/hub")
else:
    print("⚠️ 质量未达标，跳过推送")


---

## 🎓 10. 总结与最佳实践

### 完整工作流

```mermaid
graph LR
    A[修改提示词] --> B[本地测试]
    B --> C[自动捕获数据]
    C --> D[运行评估器]
    D --> E{质量达标?}
    E -->|是| F[推送到Hub]
    E -->|否| A
    F --> G[团队使用新版本]
```

### 最佳实践

1. **开发阶段**
   - 使用 `@capture_dataset` 自动收集真实数据
   - 定期同步到 LangSmith Dataset
   
2. **评估阶段**
   - 使用多个评估器全面评估
   - 设置合理的质量阈值（如 80%）
   
3. **发布阶段**
   - 评估通过后再推送到 Hub
   - 启用版本备份，便于回滚
   
4. **监控阶段**
   - 保持 LangSmith 追踪开启
   - 定期查看追踪数据和评估结果

### 快速参考

```python
# 1. 捕获数据
@capture_dataset(prompt_name="xxx", dataset_name="xxx")
def my_function():
    inputs = {...}
    capture_inputs(inputs)
    ...

# 2. 同步数据
sync_all_captures()

# 3. 评估提示词
result = prompt_manager.evaluate_prompt("prompt_name")

# 4. 推送到 Hub
prompt_manager.push("prompt_name", with_test=True, create_backup=True)
```
